In [1]:
import re

In [19]:
from PIL import Image

In [2]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/home/ubuntu/.local/lib/python3.5/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
from tesserocr import PyTessBaseAPI

In [24]:
img = 'urine20_resize800_border10.jpg'
with PyTessBaseAPI(lang='chi_sim',psm=6) as api:
    api.SetVariable('tessedit_char_whitelist','<=?+().^~\
    尿液分析胆原红素酮体血蛋白质亚硝酸盐细胞葡萄糖比重碱度镜检上皮管型结晶其他有形成份阴\
    性无结果项目名称参考区间单位颜色黄浊清～/值隐-位相电导率仅供类酵母菌粘液丝计数病理量信息完整高倍提示低潜正常浑\
    酯酶小圆非均一级标记视野平体积百阳微透明澄浅直接未见异维生外观晰个路感染态受损滴虫霉菌每鳞状柱粒草肌酐钙\
    abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
    api.SetVariable('preserve_interword_spaces','1')
#     api.SetVariable('tessedit_create_pdf','1')
#     api.ProcessPage('test', Image.open(img), 0, 'text')
    api.SetImageFile(img)
    text = api.GetUTF8Text()

In [25]:
text

'No 项目名称    结果 单位 参考区间 提示 No 项目名称    结果 单位 - 参考区间 提示\n1 尿亚硝酸盐       -         阴性 (-)    17 白细胞镜检      0    /HP 0-4. 0\n2 酸碱度 (pH)       6. 0         4. 0-8. 0     18 上皮细胞镜检      0    /AHP   0-~-1\n\n3 尿糖         -        阴性(-)    19 维生素C       0\n\n4 尿生          <          阴性 (-)     20 尿          2. 50 mmol/ 2. 57. 5\n5 微生         10. 00 nmg/LL   K=30      21 透明管型        0    /LP   0-1\n\n6 微白蛋白/尿肌酐    <3. 40 mg/mmol 3. 4       22 颜粒管型        0 0\n\n7 尿隐血         1          明性 (-)\n\n8 尿酮体          -          阴性(-)\n\n9 肌          17. 70 mmol/yL 4. 4-17. 7\n\n10 尿胆红素       -        阳性 t-)\n\n11 胆                       3. 4                     <=17\n\n12 尿比重                1. 025            1. 003-1. 09\n\n13 尿红细胞计数              1. 0        Aul        0~-22\n\n14 尿白细胞计数     0. 0   Aul   0~22\n\n15 上皮细胞计数         0. 0      Aul     0-5. 1\n\nt6 红细胞镜检                       0               /AHP          0-4. 0\n'

In [26]:
# remove space after dot
num = 0
for i in range(len(text)-1):
    if num+i >= len(text)-1:
        break
    if (text[i] == '.') & (text[1+i] == ' '):
        num += 1
        text = text[:i+1] + text[i+2:]
        
#split the text as a list of strings
text_split = text.split()

In [27]:
text_split

['No',
 '项目名称',
 '结果',
 '单位',
 '参考区间',
 '提示',
 'No',
 '项目名称',
 '结果',
 '单位',
 '-',
 '参考区间',
 '提示',
 '1',
 '尿亚硝酸盐',
 '-',
 '阴性',
 '(-)',
 '17',
 '白细胞镜检',
 '0',
 '/HP',
 '0-4.0',
 '2',
 '酸碱度',
 '(pH)',
 '6.0',
 '4.0-8.0',
 '18',
 '上皮细胞镜检',
 '0',
 '/AHP',
 '0-~-1',
 '3',
 '尿糖',
 '-',
 '阴性(-)',
 '19',
 '维生素C',
 '0',
 '4',
 '尿生',
 '<',
 '阴性',
 '(-)',
 '20',
 '尿',
 '2.50',
 'mmol/',
 '2.57.5',
 '5',
 '微生',
 '10.00',
 'nmg/LL',
 'K=30',
 '21',
 '透明管型',
 '0',
 '/LP',
 '0-1',
 '6',
 '微白蛋白/尿肌酐',
 '<3.40',
 'mg/mmol',
 '3.4',
 '22',
 '颜粒管型',
 '0',
 '0',
 '7',
 '尿隐血',
 '1',
 '明性',
 '(-)',
 '8',
 '尿酮体',
 '-',
 '阴性(-)',
 '9',
 '肌',
 '17.70',
 'mmol/yL',
 '4.4-17.7',
 '10',
 '尿胆红素',
 '-',
 '阳性',
 't-)',
 '11',
 '胆',
 '3.4',
 '<=17',
 '12',
 '尿比重',
 '1.025',
 '1.003-1.09',
 '13',
 '尿红细胞计数',
 '1.0',
 'Aul',
 '0~-22',
 '14',
 '尿白细胞计数',
 '0.0',
 'Aul',
 '0~22',
 '15',
 '上皮细胞计数',
 '0.0',
 'Aul',
 '0-5.1',
 't6',
 '红细胞镜检',
 '0',
 '/AHP',
 '0-4.',
 '0']

In [7]:
#create a dict for regular expression
urine_dict = {}
urine_dict['尿胆原'] = ['尿胆原','尿胆原\(URO\)','尿胆原\(UBG\)','尿胆原\s*URO']
urine_dict['胆红素'] = ['胆红素','尿胆红素','胆红素\s*\(BIL\)','胆红素\s*BIL']
urine_dict['酮体'] = ['酮体','尿酮体','酮体 \(KET\)','酮 体','酮体\(KET\)','酮体\s*KET']
urine_dict['隐血'] = ['血','隐血','潜血','尿隐血','尿潜血','隐血或红细胞\(BLD\)','尿潜血\(BLD\)','隐血\s*BLD']
urine_dict['尿蛋白'] = ['蛋白质','尿蛋白','蛋白质\(PRO\)','蛋白','蛋白质\s*PRO']
urine_dict['亚硝酸盐'] = ['亚硝酸盐','尿亚硝酸盐','亚硝酸盐\(NIT\)','亚硝酸\s*NIT']
urine_dict['葡萄糖'] = ['葡萄糖','尿糖','尿葡萄糖','尿糖\(GLU\)','葡萄糖\s*GLU']
urine_dict['比重'] = ['比重','尿比重','比重 \(SG\)','比 重','比重\(SG\)']
urine_dict['酸碱度'] = ['酸碱度','尿PH值','尿酸碱度','PH','酸碱度\(PH\)','尿PH','pH值\(PH\)','酸碱度\(pH\)','酸碱度\s*PH']
urine_dict['颜色'] = ['颜色','颜色\s*\(COLOR\)','尿液外观']
urine_dict['浑浊度'] = ['浊度','浑浊度','透明度\(TMD\)','透明度','尿透明度','外观','清晰度']
urine_dict['电导率'] = ['电导率','电导率\s*Cond']
urine_dict['电导率信息'] = ['电导率分级','电导率信息']
urine_dict['结晶'] = ['结晶','结晶数量','结晶标记']
urine_dict['类酵母菌'] = ['类酵母菌','类酵母细胞数量','类酵母细胞标记','酵母样菌','类酵母细胞数量\s*YLC']
urine_dict['粘液丝'] = ['粘液丝','粘液丝数量','粘液','粘液丝数量\s*MCLS']
urine_dict['细菌'] = ['细菌计数','细菌','细菌\s*BACT']
urine_dict['维生素C'] = ['VC','维生素C','维生素C\s*VC']
urine_dict['完整红细胞比率'] = ['完整红细胞比率','完整红细胞百分比','完整红细胞百分比\s*NL-RBC%']
urine_dict['完整红细胞数量'] = ['完整红细胞数量','完整红细胞绝对值','完整红细胞绝对值\s*NL-RBC#']
urine_dict['尿路感染信息'] = ['尿路感染信息']
urine_dict['尿钙'] = ['尿钙']
urine_dict['精子'] = ['精子']
urine_dict['其他有形成份'] = ['其他有形成份']
urine_dict['70%红细胞平均体积'] = ['70%红细胞平均体积','70%红细胞平均体积\s*RBC-P70']
urine_dict['白细胞(高倍视野)'] = ['白细胞\(HPF\)','白细胞\(高倍\)','每高倍视野白细胞数','白细胞\(高倍视野\)']
urine_dict['红细胞(高倍视野)'] = ['红细胞\(高倍\)','红细胞\(HPF\)','每高倍视野白细胞数','红细胞\(高倍视野\)','红细胞\s*(RBC)']
urine_dict['上皮细胞(高倍视野)'] = ['上皮细胞\(高倍视野\)','上皮细胞\(高倍\)','每高倍视野上皮细胞数']
urine_dict['红细胞信息'] = ['红细胞信息','红细胞形态信息','红细胞形态变异信息','红细胞信息\s*info']
urine_dict['红细胞镜检'] = ['直接镜检-红细胞','镜检红细胞','红细胞\(镜检\)','镜检：红细胞\(RBC\)','红细胞镜检']
urine_dict['白细胞镜检'] = ['直接镜检-白细胞','镜检白细胞','白细胞\(镜检\)','镜检：白细胞','白细胞镜检']
urine_dict['上皮细胞镜检'] = ['上皮细胞\(镜检\)','上皮细胞镜检']
urine_dict['小圆上皮细胞'] = ['小圆上皮','小圆上皮细胞','小圆上皮细胞标记','小圆上皮细胞数量','小圆细胞','上皮细胞','上皮细胞计数']
urine_dict['管型'] = ['管型','管型计数','管型计数\s*\(CAST\)','管型\s*CAST']
urine_dict['病理管型'] = ['病理管型','病理管型\s*P.CAST','病理管型标记']
urine_dict['管型低倍'] =['管型低倍','管型\(低倍视野\)','管型\(低倍\)','管型\s* \(LPF\)','每低倍视野透明管型数']
urine_dict['透明管型镜检'] = ['透明管型\(镜检\)','透明管型','镜检：透明管型','直接镜检-管型','透明管型']
urine_dict['颗粒管型镜检'] = ['颗粒管型\(镜检\)','颗粒管型','镜检：粒状管型']
urine_dict['细胞管型镜检'] = ['细胞管型\s*\(镜检\)']
# pos/neg
urine_dict['白细胞'] = ['白细胞','白细胞\(LEU\)','尿白细胞\(LEU\)','白细胞\s*WBC']
# count
urine_dict['白细胞计数'] = ['白细胞计数','白细胞计数(WBC)','白细胞\(UF\)','白细胞\(UF-1000i\)','白细胞','白细胞\(UF\)\s*WBC\(UF\)','尿白细胞计数']
urine_dict['红细胞计数'] = ['红细胞计数','红细胞计数(RBC)','红细胞计数\s*\(RBC\)','红细胞\(UF\)','红细胞','尿红细胞计数']
# sort the names according to their length, longest name first
for k, names in urine_dict.items():
    names.sort(key = lambda s: -len(s))
    
# names of the tests
urine_tests = list(urine_dict.keys())
    
# result is the dict where we store test names and values
result = {}
for test in urine_tests:
    result.setdefault(test, '')
    
    

In [8]:
# find the perfect match for test names
for k, names in urine_dict.items():
    for name in names:
        value = re.findall(''.join([name, '(\s*)(\S{1,6})']), text)
        if value:
            result[k] = value[0][1]
            break

In [9]:
result

{'70%红细胞平均体积': '',
 '上皮细胞(高倍视野)': '',
 '上皮细胞镜检': '0',
 '亚硝酸盐': '-',
 '其他有形成份': '',
 '完整红细胞数量': '',
 '完整红细胞比率': '',
 '小圆上皮细胞': '0.0',
 '尿胆原': '',
 '尿蛋白': '/尿肌酐',
 '尿路感染信息': '',
 '尿钙': '',
 '比重': '1.025',
 '浑浊度': '',
 '电导率': '',
 '电导率信息': '',
 '病理管型': '',
 '白细胞': '镜检',
 '白细胞(高倍视野)': '',
 '白细胞计数': '0.0',
 '白细胞镜检': '0',
 '管型': '0',
 '管型低倍': '',
 '类酵母菌': '',
 '粘液丝': '',
 '精子': '',
 '红细胞(高倍视野)': '',
 '红细胞信息': '',
 '红细胞计数': '1.0',
 '红细胞镜检': '0',
 '细胞管型镜检': '',
 '细菌': '',
 '结晶': '',
 '维生素C': '0',
 '胆红素': '-',
 '葡萄糖': '-',
 '透明管型镜检': '0',
 '酮体': '-',
 '酸碱度': '(pH)',
 '隐血': '1',
 '颗粒管型镜检': '',
 '颜色': ''}

In [10]:
for k, v in result.items():
    if not v:
        fuzzy = process.extractOne(k, text_split, scorer=fuzz.ratio)
        print(k, fuzzy)
        if fuzzy[1] >= 60:
            name = fuzzy[0]
            if '(' in name:
                    index = name.find('(')
                    name = name[:index] + '\\' + name[index:]
            if ')' in name:
                    index = name.find(')')
                    name = name[:index] + '\\' + name[index:]
            value = re.findall(''.join([name, '(\s*)(\S{1,6})']), text)
            if value:
                result[k] = value[0][1]

病理管型 ('透明管型', 50)
上皮细胞(高倍视野) ('上皮细胞镜检', 53)
完整红细胞比率 ('红细胞镜检', 50)
白细胞(高倍视野) ('白细胞镜检', 46)
浑浊度 ('酸碱度', 33)
尿钙 ('尿', 67)
粘液丝 ('No', 0)
尿胆原 ('尿胆红素', 57)
细胞管型镜检 ('白细胞镜检', 73)
尿路感染信息 ('尿', 29)
电导率 ('No', 0)
电导率信息 ('No', 0)
管型低倍 ('透明管型', 50)
结晶 ('结果', 50)
其他有形成份 ('No', 0)
颜色 ('颜粒管型', 33)
精子 ('No', 0)
红细胞(高倍视野) ('红细胞镜检', 46)
细菌 ('白细胞镜检', 29)
70%红细胞平均体积 ('红细胞镜检', 40)
完整红细胞数量 ('尿红细胞计数', 62)
红细胞信息 ('红细胞镜检', 60)
颗粒管型镜检 ('颜粒管型', 60)
类酵母菌 ('No', 0)


In [22]:
char_allowed = '0123456789.+-()尿路感染?红色性弱阴阳未见异常提示检出无<= 清分类正常见浅微黄澄清晰浑浊非均一型红细胞级NORMALNEGnegnormal/HP'
# in preparation for jsonify
rl = []
for k, v in result.items():
    r = {}
    r['test_name'] = k
    r['value'] = v
    if not v:
        confidence = 0
    else:
        confidence = 100
    for c in v:
        if c not in char_allowed:
            print(v, c)
            confidence = 0
            break
    r['confidence'] = confidence
    rl.append(r)
    
    
    

(pH) p
镜检 镜
亚硝酸盐 亚
/尿肌酐 肌


In [24]:
rl

[{'confidence': 0, 'test_name': '酸碱度', 'value': '(pH)'},
 {'confidence': 100, 'test_name': '酮体', 'value': '-'},
 {'confidence': 100, 'test_name': '红细胞计数', 'value': '1.0'},
 {'confidence': 0, 'test_name': '病理管型', 'value': ''},
 {'confidence': 100, 'test_name': '红细胞镜检', 'value': '0'},
 {'confidence': 100, 'test_name': '上皮细胞镜检', 'value': '0'},
 {'confidence': 100, 'test_name': '小圆上皮细胞', 'value': '0.0'},
 {'confidence': 100, 'test_name': '隐血', 'value': '1'},
 {'confidence': 0, 'test_name': '上皮细胞(高倍视野)', 'value': ''},
 {'confidence': 0, 'test_name': '完整红细胞比率', 'value': ''},
 {'confidence': 0, 'test_name': '白细胞(高倍视野)', 'value': ''},
 {'confidence': 0, 'test_name': '浑浊度', 'value': ''},
 {'confidence': 0, 'test_name': '白细胞', 'value': '镜检'},
 {'confidence': 0, 'test_name': '尿钙', 'value': '亚硝酸盐'},
 {'confidence': 0, 'test_name': '粘液丝', 'value': ''},
 {'confidence': 0, 'test_name': '尿胆原', 'value': ''},
 {'confidence': 100, 'test_name': '管型', 'value': '0'},
 {'confidence': 100, 'test_name': '细胞管型镜